## Executing queries in PL/pgSQL

---
### Why does this fail?

In [4]:
CREATE OR REPLACE FUNCTION foo() RETURNS TABLE (the_answer int)
LANGUAGE plpgsql AS $$
BEGIN
RETURN QUERY
    SELECT 42 as "The Answer";
END;    
$$;

SELECT * from foo();

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.029

the_answer
42


---
### This works!

In [5]:
CREATE OR REPLACE FUNCTION foo() RETURNS TABLE (the_answer int)
LANGUAGE plpgsql AS $$
BEGIN
    PERFORM 42 as "The Answer";
END;    
$$;

SELECT foo();

CREATE FUNCTION

(0 row(s) affected)

Total execution time: 00:00:00.015

foo


---
### Returning that value

In [6]:
CREATE OR REPLACE FUNCTION bar(OUT a INT)
LANGUAGE plpgsql AS $$
BEGIN
    SELECT 42 AS "The Answer" INTO bar.a;
END;    
$$;

SELECT bar();

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.030

bar
42


---
### INTO clause
    SELECT select_expressions INTO [STRICT] target FROM ...;
    INSERT ... RETURNING expressions INTO [STRICT] target;
    UPDATE ... RETURNING expressions INTO [STRICT] target;
    DELETE ... RETURNING expressions INTO [STRICT] target;

In [7]:
DO $$
DECLARE 
    the_answer INT;
    cust_row customers%ROWTYPE;
    firstname VARCHAR;
    lastname VARCHAR;
    new_cat INT;
    old_catname VARCHAR;
BEGIN
    SELECT 42 INTO the_answer;
    SELECT * INTO cust_row FROM customers c 
        WHERE c.customerid = 42;
    SELECT c.firstname, c.lastname INTO firstname, lastname
        FROM customers c 
        WHERE c.customerid = 42;
    INSERT INTO categories (categoryname) 
        VALUES ('MyCategory')
        RETURNING category INTO new_cat;
    DELETE FROM categories 
        WHERE category = new_cat
        RETURNING categoryname INTO old_catname;  
    RAISE NOTICE 'id: %, name: %', new_cat, old_catname; 
END       
$$

NOTICE: id: 17, name: MyCategory

DO

Total execution time: 00:00:00.058

In [8]:
CREATE OR REPLACE FUNCTION foo() RETURNS TABLE (firstname varchar)
LANGUAGE plpgsql AS $$
BEGIN
RETURN QUERY
    UPDATE customers c
        SET firstname = c.firstname
        WHERE c.customerid = 42
    RETURNING c.firstname;
END;
$$;

SELECT foo();
DO $$ BEGIN PERFORM foo(); END; $$;

: cannot change return type of existing function

Total execution time: 00:00:00.005

In [9]:
DROP FUNCTION IF EXISTS foo();
DROP FUNCTION IF EXISTS bar(OUT a INT);

DROP FUNCTION

DROP FUNCTION

Total execution time: 00:00:00.020